In [39]:
import pandas as pd

Cargamos la data necesaria

In [40]:
df_items = pd.read_parquet("../Datasets/australian_items_limpio.parquet")
df_reviews = pd.read_parquet("../Datasets/reviews_analisis_sentiment.parquet")
df_games = pd.read_parquet("../Datasets/steam_games_limpio.parquet")

sleccionamos columnas necesaria del dataframe reviews

In [41]:
df_reviews = df_reviews[["reviews_item_id", "reviews_recommend", "sentiment_analysis"]]

In [42]:
df_reviews

,reviews_item_id,reviews_recommend,sentiment_analysis
0,1250,True,2
1,251610,True,2
3,250320,True,2
4,211420,True,1
5,249130,True,1
...,...,...,...
231273,332310,True,1
231291,440,True,0
231293,304930,True,2
231419,265630,True,2


Seleccionamos las columnas del dataset games

In [43]:
df_games = df_games[["title", "id", "year"]]

In [44]:
df_games

,title,id,year
0,Lost Summoner Kitty,761140,2018.0
0,Lost Summoner Kitty,761140,2018.0
0,Lost Summoner Kitty,761140,2018.0
0,Lost Summoner Kitty,761140,2018.0
0,Lost Summoner Kitty,761140,2018.0
...,...,...,...
32132,Russian Roads,610660,2018.0
32132,Russian Roads,610660,2018.0
32132,Russian Roads,610660,2018.0
32133,EXIT 2 - Directions,658870,2017.0


Combinamos los dos dataframes

In [45]:
df_user_recomend = pd.merge(df_games, df_reviews, left_on="id", right_on="reviews_item_id", how="inner")

In [46]:
df_user_recomend

,title,id,year,reviews_item_id,reviews_recommend,sentiment_analysis
0,Half-Life,70,1998.0,70,True,1
1,Half-Life,70,1998.0,70,True,1
2,Half-Life,70,1998.0,70,True,1
3,Half-Life,70,1998.0,70,True,2
4,Half-Life,70,1998.0,70,True,2
...,...,...,...,...,...,...
100224,Counter-Strike: Condition Zero,80,2004.0,80,False,1
100225,Counter-Strike: Condition Zero,80,2004.0,80,True,2
100226,Counter-Strike: Condition Zero,80,2004.0,80,True,2
100227,Counter-Strike: Condition Zero,80,2004.0,80,True,1


In [47]:
df_user_recomend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100229 entries, 0 to 100228
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   title               100229 non-null  object 
 1   id                  100229 non-null  object 
 2   year                99716 non-null   float64
 3   reviews_item_id     100229 non-null  object 
 4   reviews_recommend   100229 non-null  bool   
 5   sentiment_analysis  100229 non-null  int64  
dtypes: bool(1), float64(1), int64(1), object(3)
memory usage: 3.9+ MB


Creamos la funcion

In [51]:
def userRecoomend(año: int):
    games_year = df_user_recomend[df_user_recomend["year"]== año]
    recommends = games_year[(games_year["reviews_recommend"]== True) & (games_year["sentiment_analysis"]== 2)]
    
    # Agrupamos por juego recomendados
    games = recommends["title"].value_counts().reset_index()
    games.columns = ["games", "recommend_count"]
    
    # Ordenamos y obtenemos el top3
    top_games = games.nlargest(3, "recommend_count")
    
    resultado = [{"Puesto {}: {}".format(i + 1, row['games']): row['recommend_count']} for i, row in top_games.iterrows()]
    
    return resultado

In [57]:
userRecoomend(2004)

[{'Puesto 1: Counter-Strike: Source': 134},
 {'Puesto 2: Half-Life 2': 104},
 {'Puesto 3: Rome: Total War™ - Collection': 11}]

Comprobamos la funcon y decidimos cargar el datasat en la carpeta data, donde se encuentra la data que consumen las funciones

In [54]:
df_user_recomend.to_parquet("../Funciones/data/userRecommend.parquet")